# 모두를 위한 딥러닝 시즌2 - TensorFlow

## LAB05-1 - Logistic Regression (Diabetes)

Logistic Classification을 diabetes data를 활용해 모델을 만들어보기!

In [16]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline

tf.compat.v1.enable_eager_execution()
tf.random.set_seed(777)
print(tf.__version__)

2.5.0


In [17]:
#dataset
xy = np.loadtxt('data_diabetes.csv', delimiter=',',dtype = np.float32)
x_train = xy[:,0:-1]
y_train = xy[:,[-1]]

print(x_train.shape, y_train.shape)
print(xy)

(759, 8) (759, 1)
[[-0.294118   0.487437   0.180328  ... -0.53117   -0.0333333  0.       ]
 [-0.882353  -0.145729   0.0819672 ... -0.766866  -0.666667   1.       ]
 [-0.0588235  0.839196   0.0491803 ... -0.492741  -0.633333   0.       ]
 ...
 [-0.411765   0.21608    0.180328  ... -0.857387  -0.7        1.       ]
 [-0.882353   0.266332  -0.0163934 ... -0.768574  -0.133333   0.       ]
 [-0.882353  -0.0653266  0.147541  ... -0.797609  -0.933333   1.       ]]


### 위 data를 기준으로 가설 검증을 통해 Logistic Classification 모델 만들기

In [18]:
#batch 사이즈는 한 번에 학습시킬 사이즈로 정하기
dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(len(x_train))

In [19]:
#초기값을 variable 0 혹은 random값으로 설정 가능하기에 0값으로 설정함.
W = tf.Variable(tf.zeros([8,1]), name='weight')
b = tf.Variable(tf.zeros([1]),name = 'bias')

### sigmoid 함수를 가설로 선언
- sigmoid는 0과 1의 값만을 리턴   
tf.sigmoid(tf.matmul(X,W)+b)

In [20]:
def logistic_regression(features):
    hypothesis  = tf.divide(1., 1. + tf.exp(tf.matmul(features, W) + b))
    return hypothesis

### 가설을 검증할 Cost 함수를 정의


In [25]:
def loss_fn(hypothesis, features, labels):
    cost = -tf.reduce_mean(labels * tf.math.log(logistic_regression(features)) + (1 - labels) * tf.math.log(1 - hypothesis))
    return cost

optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)

### 추론한 값은 0.5를 기준으로 0과 1의 값을 리턴
- sigmoid 함수를 통해 예측값이 0.5보다 크면 1을 반환하고 0.5보다 작으면 0을 반환
- 가설을 통해 실제 값과 비교한 정확도 측정

tf.cast: 텐서를 새로운 형태로 캐스팅하는 데 사용.   
tf.equal : 두 텐서의 원소를 비교해 boolean 값 출력

In [26]:
def accuracy_fn(hypothesis, labels):
    predicted = tf.cast(hypothesis > 0.5, dtype=tf.float32) #tf.cast 메서드를 이용해 0.5보다 큰 hypothesis를 입력값으로 받고 이를 float형태로 바꿔줌.
    accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted, labels), dtype=tf.int32))
    return accuracy

### GradientTape을 통해 경사값 계산

In [27]:
def grad(hypothesis, features, labels):
    with tf.GradientTape() as tape:
        loss_value = loss_fn(logistic_regression(features),features, labels)
    return tape.gradient(loss_value, [W,b])

### Tensorflow를 통한 실행을 위해 Session를 선언

Session 선언??
텐서플로우는 방향 그래프를 기반으로 해 설계된 기계학습 라이브러리.   
우리가 tf.add()등의 수행한 명령어는 흐름이 있는 그래프를 정의한 것이고 실제로 연산을 수행한 것이 아님.   
데이터를 넣어 흐름이 이루어지도록 만들어야 하고 이런 동작을 Session이 수행.
- 위의 Data를 cost 함수를 통해 학습시킨 후 모델 생성

In [28]:
EPOCHS = 1000
for step in range(EPOCHS+1):
    for features, labels in iter(dataset):
        grads = grad(logistic_regression(features), features, labels)
        optimizer.apply_gradients(grads_and_vars=zip(grads, [W,b]))
        if step%100 ==0:
            print("Iter:{}, Loss:{:.4f}".format(step, loss_fn(logistic_regression(features),features, labels)))

Iter:0, Loss:0.6924
Iter:100, Loss:0.6456
Iter:200, Loss:0.6252
Iter:300, Loss:0.6138
Iter:400, Loss:0.6057
Iter:500, Loss:0.5991
Iter:600, Loss:0.5932
Iter:700, Loss:0.5877
Iter:800, Loss:0.5826
Iter:900, Loss:0.5779
Iter:1000, Loss:0.5734
